## GridWorld
Ph.D Leonarod A, Espinosa, M.Sc Andrej Scherbakov-Parland, BIT Kristoffer Kuvaja Adolfsson

### Bibliography:

* Sutton, Richard S., and Andrew G. Barto. Reinforcement learning: An introduction. MIT press, 2018.
http://incompleteideas.net/book/bookdraft2017nov5.pdf  (chapter 4)

In [48]:
# imports
import numpy as np

In [49]:
# Utilits
def printV(V, grid):
    for idx, row in enumerate(grid.grid):
        for idy, _ in enumerate(row):            
            state = grid.m * idx + idy 
            print('%.2f' % V[state], end='\t')
        print('\n')
    print('--------------------')

def printPolicy(policy, grid):
    for idx, row in enumerate(grid.grid):
        for idy, _ in enumerate(row):            
            state = grid.m * idx + idy 
            if state in grid.stateSpace:
                string = ''.join(policy[state])
                print(string, end='\t')
            else:
                print('', end='\t')
        print('\n')
    print('--------------------')    

def printQ(Q, grid):
    for idx, row in enumerate(grid.grid):
        for idy, _ in enumerate(row):            
            state = grid.m * idx + idy            
            if state != grid.m * grid.n - 1:
                vals = [np.round(Q[state,action], 5) for action in grid.possibleActions]
                print(vals, end='\t')
        print('\n')
    print('--------------------')

def sampleReducedActionSpace(grid, action):
    actions = grid.possibleActions[:]
    actions.remove(action)
    sample = np.random.choice(actions)
    return sample

In [50]:
class WindyGrid(object):
    def __init__(self, m, n, wind):         
        self.grid = np.zeros((m,n))                            # representation of the grid
        self.m = m
        self.n = n
        self.stateSpace = [i for i in range(self.m*self.n)]        
        self.stateSpace.remove(28)                              # Terminal state
        self.stateSpacePlus = [i for i in range(self.m*self.n)] # State space + terminal state
        self.actionSpace = {'U': -self.m, 'D': self.m, 
                            'L': -1, 'R': 1}
        self.possibleActions = ['U', 'D', 'L', 'R']
        self.agentPosition = 0
        self.wind = wind

    def isTerminalState(self, state):
        return state in self.stateSpacePlus and state not in self.stateSpace 

    def getAgentRowAndColumn(self):                               # position of agent
        x = self.agentPosition // self.m
        y = self.agentPosition % self.n
        return x, y
    
    def setState(self, state):
        x, y = self.getAgentRowAndColumn() 
        self.grid[x][y] = 0            
        self.agentPosition = state        
        x, y = self.getAgentRowAndColumn() 
        self.grid[x][y] = 1   
    
    def offGridMove(self, newState, oldState):
        # if we move into a row not in the grid
        if newState not in self.stateSpacePlus:
            return True
        # if we're trying to wrap around to next row
        elif oldState % self.m == 0 and newState  % self.m == self.m - 1:
            return True
        elif oldState % self.m == self.m - 1 and newState % self.m == 0:
            return True
        else:
            return False   
        
    # Include wind stenght.
    def step22(self, action):
        agentX, agentY = self.getAgentRowAndColumn()
        if agentY >= 0 and agentY < len(self.wind):  # Kontrollera om agentY är ett giltigt index
            resultingState = self.agentPosition + self.actionSpace[action] + self.wind[agentY] * self.actionSpace['U']
            if resultingState < 0:  # If the wind is trying to push the agent off the grid
                resultingState += self.m

        if agentX > 0:
            resultingState = self.agentPosition + self.actionSpace[action] + \
                            self.wind[agentY] * self.actionSpace['U']
            if resultingState < 0: #if the wind is trying to push agent off grid
                resultingState += self.m
        else:
            if action == 'L' or action == 'R':
                resultingState = self.agentPosition + self.actionSpace[action]
            else:
                resultingState = self.agentPosition + self.actionSpace[action] + \
                            self.wind[agentY] * self.actionSpace['U']
        #reward = -1 if not self.isTerminalState(resultingState) else 0
        reward = -1
        if not self.offGridMove(resultingState, self.agentPosition):
            self.setState(resultingState)
            return resultingState, reward, self.isTerminalState(resultingState), None
        else:
            return self.agentPosition, reward, self.isTerminalState(self.agentPosition), None
        
    def step(self, action):
        agentX, agentY = self.getAgentRowAndColumn()

        if agentY >= 0 and agentY < len(self.wind):  # Kontrollera om agentY är ett giltigt index
            wind_effect = self.wind[agentY] * self.actionSpace['U']
        else:
            wind_effect = 0

        if agentX > 0:
            resultingState = self.agentPosition + self.actionSpace[action] + wind_effect
        else:
            if action == 'L' or action == 'R':
                resultingState = self.agentPosition + self.actionSpace[action]
            else:
                resultingState = self.agentPosition + self.actionSpace[action] + wind_effect

        if resultingState < 0:  # If the wind or action is trying to push the agent off the grid
            resultingState += self.m

        reward = -1

        if not self.offGridMove(resultingState, self.agentPosition):
            self.setState(resultingState)
            return resultingState, reward, self.isTerminalState(resultingState), None
        else:
            return self.agentPosition, reward, self.isTerminalState(self.agentPosition), None

    def reset(self):
        self.agentPosition = 0
        self.grid = np.zeros((self.m,self.n))
        return self.agentPosition, False


    def render(self):
        print('------------------------------------------')
        for row in self.grid:
            for col in row:
                if col == 0:
                    print('-', end='\t')
                elif col == 1:
                    print('X', end='\t')
            print('\n')
        print('------------------------------------------')
        

In [51]:
grid = WindyGrid(8,8, wind=[0, 0, 1, 2, 1, 0])
grid.render()

------------------------------------------
-	-	-	-	-	-	-	-	

-	-	-	-	-	-	-	-	

-	-	-	-	-	-	-	-	

-	-	-	-	-	-	-	-	

-	-	-	-	-	-	-	-	

-	-	-	-	-	-	-	-	

-	-	-	-	-	-	-	-	

-	-	-	-	-	-	-	-	

------------------------------------------


## First visit Monte Carlo Prediction

In [52]:
def MC_first_visit(X =6 ,loop=500,Y=1):
    print("GAMMA:",Y)
    print("Size:",X,X)

    grid = WindyGrid(X,X, wind=[0, 0, 1, 2, 1, 0])
    GAMMA = 1.0
    if(Y):
        GAMMA = Y
    

    policy = {}                              #  a dictionary that maps each
    for state in grid.stateSpace:            #  state to the list of possible actions
        policy[state] = grid.possibleActions

    V = {}                                   # Initialize our initial estimate of the value
    for state in grid.stateSpacePlus:        # function. Each state gets a value of 0.
        V[state] = 0                                                              

    returns = {}                             #Initialize a dictionary that keeps a list
    for state in grid.stateSpace:            #of the returns for each state.
        returns[state] = []

    for i in range(loop):                     # Loop over 500 games,  
        observation, done = grid.reset()     # resetting the grid and memory with each game.
        memory = []                          # empty list to keep track of the states visited 
        statesReturns = []                   # and returns at each time step
        if i % 100 == 0:                     # Just to know if the game is running.
            print('starting episode', i)
        while not done:                      # While the game isn't done 
            # attempt to follow the policy. In this case choose an action 
            # according to the random equiprobable strategy.
            action = np.random.choice(policy[observation])    
            observation_, reward, done, info = grid.step(action)  # Take that action, get new state, reward and done
            memory.append((observation, action, reward))
            observation = observation_

        # append terminal state
        memory.append((observation, action, reward))

        G = 0                                  # set G=0
        last = True                            # initialize a Boolean to keep track of the visit to the last state                   
        for state, action, reward in reversed(memory): 
            if last:
                last = False
            else:                                    # Skip the terminal state and append the set of states
                statesReturns.append((state,G))      #  and returns to the statesReturns list. 
            G = GAMMA*G + reward

        statesReturns.reverse()                  # to ge it in chronological order
        statesVisited = []                       # keep track of the visited states during the episode.
        for state, G in statesReturns:
            if state not in statesVisited:       # Iterate over the episode and see 
                returns[state].append(G)         # if each state has been visited before. 
                V[state] = np.mean(returns[state]) 
                statesVisited.append(state)
                
                #If it hasn't, meaning this is the agent's first visit, go ahead and append 
                #the returns to the returns dictionary for that state.
                #Calculate the value function by taking the mean of the returns for that state, and finally, 
                #append that state to the list of statesVisited. 
    print("\n") 
    printV(V, grid)

## Del 1:

- Använd *first visit* Monte Carlo Metoden

1. Öka vindstyrkan med en enhet.
    - Hur ändras slutvärdesfunktionen?


2. Hur ändras värdefunktion om man ändra gamma till:
    - 𝛾=0.5
    - 𝛾=0,9
    - 𝛾=0,95


3. Testa rutnätsvärlden i storlekarna:
    - 8x8
        - Ändra på vinden, vad händer med värdefunktion?
        - Prova med 𝛾=0,9, vad händer med värdefunktion?
    - 10x10
        - Ändra på vinden, vad händer med värdefunktion?
        - Prova med 𝛾=0,9, vad händer med värdefunktion?

In [53]:
MC_first_visit(6,500,0.5)
MC_first_visit(6,500,0.9)
MC_first_visit(6,500,0.95)

GAMMA: 0.5
Size: 6 6
starting episode 0
starting episode 100
starting episode 200
starting episode 300
starting episode 400


-2.00	-2.00	-2.00	-2.00	-2.00	-2.00	

-2.00	-2.00	-2.00	-2.00	-2.00	-2.00	

-2.00	-2.00	-2.00	-2.00	-2.00	-1.99	

-2.00	-2.00	-2.00	-2.00	-2.00	-1.95	

-2.00	-2.00	-2.00	-2.00	0.00	-1.69	

-2.00	-2.00	-2.00	0.00	-1.96	-1.95	

--------------------
GAMMA: 0.9
Size: 6 6
starting episode 0
starting episode 100
starting episode 200
starting episode 300
starting episode 400


-10.00	-9.99	-9.98	-9.98	-9.96	-9.93	

-10.00	-10.00	-9.99	-9.98	-9.94	-9.87	

-10.00	-10.00	-9.97	-9.99	-9.96	-9.61	

-9.99	-10.00	-10.00	-9.96	-9.91	-8.76	

-9.99	-9.99	-9.98	-9.95	0.00	-6.18	

-9.99	-9.99	-9.98	0.00	-8.90	-8.25	

--------------------
GAMMA: 0.95
Size: 6 6
starting episode 0
starting episode 100
starting episode 200
starting episode 300
starting episode 400


-19.95	-19.95	-19.93	-19.89	-19.84	-19.68	

-19.94	-19.95	-19.95	-19.91	-19.73	-19.47	

-19.95	-19.95	-19.93	-19.76	-19.

In [54]:
MC_first_visit(8,500,0.9)

GAMMA: 0.9
Size: 8 8
starting episode 0
starting episode 100
starting episode 200
starting episode 300
starting episode 400


-9.99	-9.98	-9.97	-9.95	-9.93	-9.87	-9.80	-9.79	

-9.99	-9.99	-9.96	-9.95	-9.87	-9.70	-9.68	-9.75	

-9.98	-9.99	-9.96	-9.94	-9.88	-9.09	-9.34	-9.55	

-9.97	-9.96	-9.96	-9.96	0.00	-6.89	-8.73	-9.26	

-9.93	-9.92	-9.93	-9.95	-8.88	-8.72	-9.06	-9.31	

-9.92	-9.86	-9.97	-8.53	-6.82	-8.79	-9.39	-9.56	

-9.93	-9.69	-9.44	-9.68	-8.98	-9.05	-9.38	-9.53	

-9.98	-9.92	-9.85	0.00	-8.53	-9.07	-9.24	-9.56	

--------------------


In [55]:
MC_first_visit(10,500,0.9)

GAMMA: 0.9
Size: 10 10
starting episode 0
starting episode 100
starting episode 200
starting episode 300
starting episode 400


-9.97	-9.95	-9.93	-9.89	-9.77	-9.49	-8.93	-8.35	-7.73	-7.83	

-9.96	-9.96	-9.95	-9.89	-9.73	-9.39	-8.70	-7.63	-6.09	-7.05	

-9.97	-9.96	-9.94	-9.90	-9.82	-9.29	-8.39	-6.07	0.00	-5.85	

-9.97	-9.97	-9.95	-9.86	-9.74	-9.54	-8.87	-7.79	-6.10	-7.04	

-9.97	-9.97	-9.92	-9.90	-9.78	-9.58	-9.26	-8.83	-8.42	-8.04	

-9.98	-9.96	-9.94	-9.80	-9.78	-9.75	-9.60	-9.63	-9.19	-9.09	

-9.99	-9.99	-9.97	-9.99	-9.94	-9.92	-9.73	-9.65	-9.64	-9.72	

-9.99	-9.99	-9.99	-9.87	-9.96	-9.95	-9.82	-9.82	-9.83	-9.79	

-9.98	-9.98	-9.93	-9.09	-9.98	-9.97	-9.86	-9.96	-9.84	-9.93	

-10.00	-9.98	-9.99	0.00	-9.57	-9.76	-9.82	-9.81	-9.67	-9.91	

--------------------


## Exploring Start Monte Carlo

In [62]:
def MC_exploring_starts(X =6,Y=1):
    grid = WindyGrid(X ,X, wind=[0, 0, 1, 2, 1, 0])
    GAMMA = Y
    print("GAMMA:",Y)
    print("Size:",X,X)
   
    # Initialize Q, returns, and pairs visited
    Q = {}          
    returns = {}
    pairsVisited = {}
    for state in grid.stateSpacePlus:
        for action in grid.possibleActions:
            Q[(state, action)] = 0
            returns[(state,action)] = 0
            pairsVisited[(state,action)] = 0
    
    # initialize a random policy
    policy = {}
    for state in grid.stateSpace:
        policy[state] = np.random.choice(grid.possibleActions)
    
    for i in range(1000000):  
        if i % 50000 == 0:
            print('starting episode', i)
        statesActionsReturns = []
        observation = np.random.choice(grid.stateSpace)
        action = np.random.choice(grid.possibleActions)
        grid.setState(observation)
        observation_, reward, done, info = grid.step(action)
        memory = [(observation, action, reward)]
        steps = 1
        while not done:
            action = policy[observation_]
            steps += 1
            observation, reward, done, info = grid.step(action)
            if steps > 15 and not done:
                done = True
                reward = -steps
            memory.append((observation_, action, reward))
            observation_ = observation

        # append the terminal state
        memory.append((observation_, action, reward))
        
        G = 0        
        last = True # start at t = T - 1
        for state, action, reward in reversed(memory):
            if last:
                last = False  
            else:
                statesActionsReturns.append((state,action, G))
            G = GAMMA*G + reward

        statesActionsReturns.reverse()
        statesAndActions = []
        for state, action, G in statesActionsReturns:
            if (state, action) not in statesAndActions:
                pairsVisited[(state,action)] += 1
                returns[(state,action)] += (1 / pairsVisited[(state,action)])*(G-returns[(state,action)])                   
                Q[(state,action)] = returns[(state,action)]
                statesAndActions.append((state,action))
                values = np.array([Q[(state,a)] for a in grid.possibleActions])
                best = np.argmax(values)
                policy[state] = grid.possibleActions[best]
            
    printQ(Q, grid)
    printPolicy(policy,grid)

## Del 2


- Använd  *exploring starts* Monte Carlo Metoden

1. Öka vindstyrkan med en enhet.
    - Hur ändras slutvärdesfunktionen?


2. Hur ändras policyn om man ändra gamma till:
    - 𝛾=0.5
    - 𝛾=0,9
    - 𝛾=0,95


3. Testa rutnätsvärlden i storlekarna:
    - 8x8
        - Ändra på vinden, vad händer med policyn?
        - Prova med 𝛾=0,9, vad händer med policyn?
    - 10x10
        - Ändra på vinden, vad händer med policyn?

In [63]:
MC_exploring_starts(6,0.5)
MC_exploring_starts(6,0.9)
MC_exploring_starts(6,0.95)

GAMMA: 0.5
Size: 6 6
starting episode 0
starting episode 50000
starting episode 100000
starting episode 150000
starting episode 200000
starting episode 250000
starting episode 300000
starting episode 350000
starting episode 400000
starting episode 450000
starting episode 500000
starting episode 550000
starting episode 600000
starting episode 650000
starting episode 700000
starting episode 750000
starting episode 800000
starting episode 850000
starting episode 900000
starting episode 950000
[-1.99905, -1.9991, -1.99906, -1.99805]	[-1.99814, -1.99809, -1.99908, -1.9961]	[-1.99618, -1.99651, -1.9981, -1.99219]	[-1.99223, -1.99221, -1.99629, -1.98438]	[-1.98445, -1.9844, -1.99227, -1.96875]	[-1.96876, -1.9375, -1.98441, -1.96878]	

[-1.99904, -1.99904, -1.99904, -1.99805]	[-1.99808, -1.99806, -1.99905, -1.9961]	[-1.99616, -1.99611, -1.9981, -1.99219]	[-1.9922, -1.99226, -1.99611, -1.98438]	[-1.98438, -1.98439, -1.99222, -1.96875]	[-1.96881, -1.875, -1.98439, -1.93754]	

[-1.99905, -1.99903

In [64]:
MC_exploring_starts(8,0.9)

GAMMA: 0.9
Size: 8 8
starting episode 0
starting episode 50000
starting episode 100000
starting episode 150000
starting episode 200000
starting episode 250000
starting episode 300000
starting episode 350000
starting episode 400000
starting episode 450000
starting episode 500000
starting episode 550000
starting episode 600000
starting episode 650000
starting episode 700000
starting episode 750000
starting episode 800000
starting episode 850000
starting episode 900000
starting episode 950000
[-6.55365, -6.53187, -6.56338, -6.15475]	[-6.15981, -6.2173, -6.55077, -5.69879]	[-5.70101, -5.71642, -6.17506, -5.21892]	[-5.29169, -5.24708, -5.70339, -4.68593]	[-4.69123, -4.72812, -5.22516, -4.09565]	[-4.10798, -3.43904, -4.7223, -4.6884]	[-4.68855, -4.09555, -4.10353, -5.22253]	[-5.22464, -4.69131, -4.68677, -5.22215]	

[-6.54884, -6.14597, -6.53487, -6.14115]	[-6.15615, -5.76407, -6.53287, -5.7231]	[-5.69863, -5.7145, -6.16968, -5.21824]	[-5.23023, -5.23858, -5.71346, -4.68675]	[-4.68885, -4.69

In [65]:
MC_exploring_starts(10,0.9)

GAMMA: 0.9
Size: 10 10
starting episode 0
starting episode 50000
starting episode 100000
starting episode 150000
starting episode 200000
starting episode 250000
starting episode 300000
starting episode 350000
starting episode 400000
starting episode 450000
starting episode 500000
starting episode 550000
starting episode 600000
starting episode 650000
starting episode 700000
starting episode 750000
starting episode 800000
starting episode 850000
starting episode 900000
starting episode 950000
[-6.96055, -6.94275, -6.97612, -6.60118]	[-6.61726, -6.7311, -6.96941, -6.13788]	[-6.2699, -6.16807, -6.71008, -5.69731]	[-5.92973, -5.77033, -6.15766, -5.21719]	[-5.22574, -5.39427, -5.73117, -4.68578]	[-4.69057, -4.09516, -5.25862, -4.1071]	[-4.10013, -3.45615, -4.6907, -3.43993]	[-3.44489, -2.72003, -4.10015, -2.7111]	[-2.71518, -1.9, -3.44433, -3.44385]	[-3.44417, -2.71611, -2.71, -3.4437]	

[-6.96533, -7.0623, -6.96877, -6.5314]	[-6.6593, -6.79188, -6.93522, -6.13078]	[-6.15202, -6.17804, -6.6

## On-policy first visit Monte Carlo for $\varepsilon$-soft policies

In [68]:
def MC_without_exploring_starts(X =6,Y=0.9):
    grid = WindyGrid(X,X, wind=[0, 0, 1, 2, 1, 0])
    GAMMA = Y
    EPS = 0.4

    Q = {}
    returns = {}
    pairsVisited = {}
    for state in grid.stateSpacePlus:
        for action in grid.actionSpace.keys():
            Q[(state, action)] = 0
            returns[(state,action)] = 0
            pairsVisited[(state,action)] = 0

    policy = {}
    for state in grid.stateSpace:
        policy[state] = grid.possibleActions

    for i in range(1000000):
        statesActionsReturns = []
        if i % 100000 == 0:
            print('starting episode', i)
        observation, done = grid.reset()       
        memory = []
        steps = 0
        while not done:       
            if len(policy[observation]) > 1:
                action = np.random.choice(policy[observation])
            else:
                action = policy[observation]
            observation_, reward, done, info = grid.step(action)
            steps += 1
            if steps > 25 and not done:
                done = True
                reward = -steps
            memory.append((observation, action, reward))
            observation = observation_

        #append the terminal state
        memory.append((observation, action, reward))

        G = 0        
        last = True # start at t = T - 1
        for state, action, reward in reversed(memory):                                    
            if last:
                last = False
            else:
                statesActionsReturns.append((state,action,G))           
            G = GAMMA*G + reward
        statesActionsReturns.reverse()

        statesAndActions = []
        for state, action, G in statesActionsReturns:
            if (state, action) not in statesAndActions:
                pairsVisited[(state,action)] += 1
                returns[(state,action)] += (1 / pairsVisited[(state,action)])*(G-returns[(state,action)])                   
                Q[(state,action)] = returns[(state,action)]
                statesAndActions.append((state,action))
                values = np.array([Q[(state,a)] for a in grid.possibleActions])
                best = np.random.choice(np.where(values==values.max())[0])                    
                rand = np.random.random()
                if rand < 1 - EPS:
                    policy[state] = grid.possibleActions[best]
                else:                        
                    policy[state] = np.random.choice(grid.possibleActions)

    printQ(Q, grid)
    printPolicy(policy,grid)

## Del 3
- Använd *without exploring starts* Monte Carlo Metoden

1. Öka vindstyrkan med en enhet.
    - Hur ändras slutvärdesfunktionen?


2. Hur ändras policyn om man ändra gamma till:
    - 𝛾=0.5
    - 𝛾=0,9
    - 𝛾=0,95


3. Testa rutnätsvärlden i storlekarna:
    - 8x8
        - Ändra på vinden, vad händer med policyn?
        - Prova med 𝛾=0,9, vad händer med policyn?
    - 10x10
        - Ändra på vinden, vad händer med policyn?

In [69]:
MC_without_exploring_starts(6,0.5)
MC_without_exploring_starts(6,0.9)
MC_without_exploring_starts(6,0.95)


starting episode 0
starting episode 100000
starting episode 200000
starting episode 300000
starting episode 400000
starting episode 500000
starting episode 600000
starting episode 700000
starting episode 800000
starting episode 900000
[-2.0, -2.0, -2.0, -2.0]	[-2.00001, -2.00001, -2.00001, -2.00001]	[-2.00006, -2.00005, -2.00006, -2.00006]	[-2.00008, -2.00009, -2.00008, -2.0001]	[-2.00024, -2.00041, -2.00019, -2.00069]	[-2.00176, -3.15643, -2.03088, -2.02934]	

[-2.0, -2.0, -2.0, -2.0]	[-2.00001, -2.00001, -2.00001, -2.00001]	[-2.00002, -2.00002, -2.00002, -2.00002]	[-2.05281, -2.00027, -2.00036, -2.00018]	[-2.00071, -2.00085, -6.80045, -2.00089]	[-2.00042, -2.00107, -14.33371, -2.00032]	

[-2.00001, -2.00001, -2.00001, -2.00001]	[-2.00002, -2.00002, -2.00002, -2.00002]	[-2.00006, -2.00007, -2.00006, -2.00006]	[-2.00026, -2.00022, -2.00018, -2.0002]	[-2.00113, -2.00113, -2.00072, -2.00113]	[-1.98438, -1.75, -1.99976, -2.00452]	

[-2.00004, -2.00004, -2.00004, -2.00004]	[-2.00004, -2.00

In [70]:
MC_without_exploring_starts(8,0.5)

starting episode 0
starting episode 100000
starting episode 200000
starting episode 300000
starting episode 400000
starting episode 500000
starting episode 600000
starting episode 700000
starting episode 800000
starting episode 900000
[-2.0, -1.99992, -2.0, -1.99976]	[-2.00001, -1.9998, -2.00001, -1.99935]	[-2.00008, -2.00005, -2.00007, -1.99818]	[-2.00044, -2.00003, -2.00004, -1.99476]	[-2.00007, -2.00072, -2.00008, -1.98451]	[-2.00021, -1.95565, -2.00026, -1.99357]	[-2.00035, -1.98254, -2.00034, -1.99899]	[-2.00098, -1.99733, -2.08633, -2.00165]	

[-2.0, -1.99997, -2.0, -1.99978]	[-1.99994, -1.9998, -2.00003, -1.9994]	[-1.99954, -2.00005, -1.99999, -1.99847]	[-2.00526, -2.00161, -2.00462, -1.99841]	[-2.004, -2.002, -2.00356, -2.00558]	[-2.00034, -1.87481, -2.00032, -1.98759]	[-2.0006, -1.95408, -1.97496, -1.99745]	[-2.07366, -1.98644, -2.07395, -2.00279]	

[-2.00001, -2.00087, -2.00001, -2.00007]	[-2.00016, -2.00004, -2.00064, -1.99934]	[-1.99999, -2.00061, -2.00055, -1.99906]	[-2.02

In [71]:
MC_without_exploring_starts(10,0.5)

starting episode 0
starting episode 100000
starting episode 200000
starting episode 300000
starting episode 400000
starting episode 500000
starting episode 600000
starting episode 700000
starting episode 800000
starting episode 900000
[-2.0, -1.99995, -2.0, -1.99986]	[-2.00001, -1.99986, -2.00001, -1.99962]	[-2.00038, -2.00001, -2.00004, -1.99894]	[-2.00059, -2.00008, -2.00026, -1.9969]	[-2.0001, -2.0001, -2.00021, -1.99111]	[-2.00013, -1.97935, -2.00015, -1.9775]	[-2.00024, -1.94854, -2.00024, -1.94317]	[-2.0005, -1.85604, -2.00051, -1.87188]	[-2.00189, -1.76744, -2.0021, -1.95507]	[-2.00452, -1.91031, -2.00333, -2.00453]	

[-2.00001, -1.99998, -2.00001, -1.99986]	[-1.99994, -1.99995, -2.00001, -1.9996]	[-1.99965, -2.00381, -1.99995, -1.99892]	[-2.00126, -2.0041, -2.00106, -2.00834]	[-2.00255, -2.00112, -2.00534, -2.16535]	[-2.00045, -1.95379, -2.00866, -1.95343]	[-1.99278, -1.87339, -1.99944, -1.88967]	[-2.0007, -1.64985, -1.95934, -1.64709]	[-2.00023, -1.0, -1.99964, -1.87946]	[-2.0

## Off-Policy Monte Carlo prediction

In [59]:
def MC_off_policy_prediction():
    grid = WindyGrid(6,6, wind=[0,0,1,2,1,0])
    GAMMA = 0.9

    Q = {}
    C = {}
    for state in grid.stateSpacePlus:
        for action in grid.possibleActions:
            Q[(state,action)] = 0
            C[(state,action)] = 0
    
    targetPolicy = {}
    for state in grid.stateSpace:
        targetPolicy[state] = np.random.choice(grid.possibleActions)

    for i in range(1000000):
        if i % 100000 == 0:
            print(i)            
        behaviorPolicy = {}
        for state in grid.stateSpace:
            behaviorPolicy[state] = grid.possibleActions
        memory = []
        observation, done = grid.reset()
        steps = 0
        while not done:
            action = np.random.choice(behaviorPolicy[observation])
            observation_, reward, done, info = grid.step(action)
            steps += 1
            if steps > 25:
                done = True
                reward = -steps
            memory.append((observation, action, reward))
            observation = observation_
        memory.append((observation, action, reward))
        
        G = 0
        W = 1
        last = True
        for (state, action, reward) in reversed(memory):            
            if last:
                last = False
            else:
                C[state,action] += W
                Q[state,action] += (W / C[state,action])*(G-Q[state,action])
                prob = 1 if action in targetPolicy[state] else 0
                W *= prob/(1/len(behaviorPolicy[state]))
                if W == 0:
                    break
            G = GAMMA*G + reward
    printQ(Q, grid)
    printPolicy(targetPolicy,grid)

## Del 4
- Använd *off-policy prediction* Monte Carlo Metoden

1. Öka vindstyrkan med en enhet.
    - Hur ändras slutvärdesfunktionen?


2. Hur ändras policyn om man ändra gamma till:
    - 𝛾=0.5
    - 𝛾=0,9
    - 𝛾=0,95


3. Testa rutnätsvärlden i storlekarna:
    - 8x8
        - Ändra på vinden, vad händer med policyn?
        - Prova med 𝛾=0,9, vad händer med policyn?
    - 10x10
        - Ändra på vinden, vad händer med policyn?